## Merge historical and real-time data


Function of script:
* Get latest valid date of historical water level data
* Retain real-time data only after this date
* Concatenate historical data and real-time data into one file

Context in workflow:
* If there is no real-time data for a given station, the historical data will be copied to the next folder (essentially skipping the merge step)

In [10]:
import os
import json
import shutil
import urllib.request
import pandas as pd
import pytz
from pathlib import Path

DATA = Path("data")
INPUTS = DATA / "inputs"
OUTPUTS = DATA / "outputs"

stnlist_csv = INPUTS / "metadata.csv"
input_dir_old = INPUTS / "wl_historical"
input_dir_new = OUTPUTS / "wl_realtime"
output_dir = OUTPUTS / "wl_merged"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

#### Step 1 - Get list of real-time stations on the CHS API

In [11]:
with urllib.request.urlopen('https://api-iwls.dfo-mpo.gc.ca/api/v1/stations') as url:
    data = json.loads(url.read().decode())
chs_api_stns = pd.DataFrame(data)

#### Step 2 - Add latest years to historical data

In [12]:
def get_tz_from_str(tz_str):
    """Converts a timezone string (e.g., 'UTC-4' or 'UTC-3:30') to a pytz timezone object."""
    if tz_str.startswith("UTC"):
        offset = tz_str[3:]
        hours, minutes = map(int, offset.split(':')) if ':' in offset else (int(offset), 0)
        return pytz.FixedOffset(hours * 60 - minutes)
    return None

stnlist = pd.read_csv(stnlist_csv, encoding='latin1')

for i, row in stnlist.iterrows():
    stn_num = str(row['stn_num']).zfill(5)
    stn_name = row['stn_name']
    tz = get_tz_from_str(row['lcl_stnd_tz'])
    file = os.path.join(input_dir_new, f'{stn_num}_realtime_wl.csv')
    old_data = os.path.join(input_dir_old, f'{stn_num}_HOURLY.DAT')
    all_data = os.path.join(output_dir, f'{stn_num}_HOURLY.DAT')

    # Assume that missing files means that station didn't have any real-time data,
    if not os.path.exists(file):
        shutil.copy(old_data, all_data)
        continue

    new_data = pd.read_csv(file, index_col=0, parse_dates=True)

    with open(old_data, "r", encoding="latin1") as f:
        lines = f.readlines()

    latest_lst = None
    for line in reversed(lines):
        parts = line.strip().split()
        if len(parts) == 3:
            date, time, value = parts
            try:
                value = float(value)
                if value != 999.999:
                    latest_lst = f"{date} {time}"
                    break
            except ValueError:
                continue

    latest_lst_dt = pd.to_datetime(latest_lst, format='%Y/%m/%d %H:%M')
    latest_lst_dt = tz.localize(latest_lst_dt)
    latest_utc_dt = latest_lst_dt.astimezone(pytz.utc)
    new_data_cropped = new_data[new_data.index > latest_utc_dt]

    new_lines = []
    for timestamp, row in new_data_cropped.iterrows():
        event_time = timestamp.to_pydatetime()
        date_str = event_time.strftime("%Y/%m/%d")
        time_str = event_time.strftime("%H:%M")
        value_str = f"{row['value']:7.3f}"
        new_lines.append(f"{date_str} {time_str} {value_str}\n")

    with open(all_data, "w", encoding="latin1") as f:
        f.writelines(lines)
        f.writelines(new_lines)